# This is the code which solves the Static Traffic Assignment Problem

In [1]:
import numpy as np

In [2]:
network = 'data/braess'

## 1. We load the graph and the demand
Both graph and demand are in csv file

In [3]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [214]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])
demand[0][0] = 10
print(demand)

[[ 10.   3. 100.]]


Then store the links in a dictionary

$\underline{\text{Edit:}}$ we don't need it anymore, we store the graph adjacency matrix as a sparse matrix

In [16]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    
# print(graph_dict)
# Here we remove graph_dict
#del graph_dict

Then, we define the function which gives the travel time as a function of the flow

In [112]:
# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [float('inf') for i in range(len(f))] # this might be to much, to do once we have everything done
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else float('inf') for i in range(len(f))]
    return tt_tmp

In [113]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
print(nb_nodes)

4


## 2. We compute the all or nothing flow allocation

In [7]:
import scipy

In [8]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [181]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        # G[int(graph[i][1])].getcol(int(graph[i][2])).toarray().reshape(1)[0]
        # G[int(graph[i][1])][int(graph[i][2])] = tt[i]
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]

In [182]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links))) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)
print(G)

  (0, 1)	1.0
  (0, 2)	2.0
  (1, 2)	0.25
  (1, 3)	2.0
  (2, 3)	1.0


In [183]:
class path:
    def __init__(self,links):
        self.links = links
        self.flow = 0
    def add_flow(self, flow):
        self.flow += flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.flow) + " is on " + str(self.links) 

In [184]:
paths_used = {}
print(paths_used)
p = path([0, 1, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 1, 1, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 0, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)

{}
{430: (<__main__.path object at 0x113f9eac8>, 0)}
{430: (<__main__.path object at 0x113f9eac8>, 0), 405: (<__main__.path object at 0x113f9e6d8>, 0)}
{430: (<__main__.path object at 0x113f9eac8>, 0), 405: (<__main__.path object at 0x113f9e6d8>, 0), 425: (<__main__.path object at 0x113f9e630>, 0)}


Now let's compute the all or nothing allocation

In [199]:
# computing the all or nothing flow
paths_used = {}
i = 0

def all_or_nothing():
    faon = np.zeros(nb_links)
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    """faon = np.zeros(shape = nb_links)"""
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp = d_tmp
        links = []
        while node_tmp != o_tmp:
            node_tmp_d = return_predecessors[o_tmp][node_tmp]
            link_tmp = list(G.todok().keys()).index((node_tmp_d, node_tmp)) #I don't know how much time it takes, should I stock the list of keys in memory?
            links.insert(0, link_tmp)
            faon[link_tmp] += flow_tmp
            node_tmp = node_tmp_d
        
        p = path(links)
        p.add_flow(flow_tmp)
        print(p)
        if hash(p) in paths_used:
            continue
            # print(str(p) + " is already in paths_used" + str(paths_used[hash(p)]))
        else:
            paths_used[hash(p)] = (p, i)
            i = i+1
    
        # Here store the path
        # p = np.zeros()
        # next is for the link flow
        """
        node_tmp_d = d_tmp
        while node_tmp_d != o_tmp:
            node_tmp = return_predecessors[o_tmp][node_tmp_d]
            link_tmp = int(graph_dict[node_tmp][node_tmp_d])
            faon[link_tmp] += flow_tmp
            node_tmp_d = node_tmp
    return faon
    """
    return faon

We define the line search

In [200]:
all_or_nothing()
print(paths_used)

100.0 is on [1, 4]
{21: (<__main__.path object at 0x113dd72b0>, 0)}


In [201]:
# TO DO: rewrite this function
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [209]:
eps=1e-8
f = all_or_nothing()
print(f)
update_travel_time(travel_time(f))

for i in range(1000):
    if i % 100 == 0:
        print(i)
    faon = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    if(i%100==0):
        print(i)
        print(s)
    if s < eps:
        break
    f = (1. - s) * f + s * faon
    update_travel_time(travel_time(f))

100.0 is on [1, 4]
[  0. 100.   0.   0. 100.]
0
100.0 is on [0, 3]
0
0.5
100.0 is on [1, 4]


In [212]:
print(G)
print(f)

  (0, 1)	6.0
  (0, 2)	2.0
  (1, 2)	0.25
  (1, 3)	2.0
  (2, 3)	6.0
[50. 50.  0. 50. 50.]


In [211]:
update_travel_time(travel_time(f))